In [2]:
!pip install sentence-transformers
import pandas as pd
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
import math
import logging
from datetime import datetime
import os
from collections import defaultdict
from datetime import datetime
import torch

In [3]:
def divide_into_par(text, threshold=10):
  splits = text.split('\n')
  pars = []
  par = ''
  for split in splits:
    words = split.split(' ')
    if len(words) > 0 and len(words) < threshold:
      if len(par) > 0:
        pars.append(par)
        par = ''
    elif len(words) > threshold:
      par += split
  if len(pars) > 0:
    return pars
  return [text]

In [4]:
wiki = pickle.load( open( '/content/drive/MyDrive/data/wikipedia2.pickle', 'rb' ))
wiki = pd.DataFrame(data=wiki, columns=['id', 'url', 'title', 'text'])
wiki['paragraph'] = wiki['text'].apply(divide_into_par)
wiki.reset_index(drop=True, inplace=True)
wiki = wiki.explode('paragraph')
wiki.reset_index(drop=True, inplace=True)
wiki = wiki.sort_values(by='title')
wiki.reset_index(drop=True, inplace=True)



In [8]:
wiki.iloc[250000]

id                                                      732262
url                 http://pl.wikipedia.org/wiki/?curid=732262
title                                               Compostela
text         Compostela - rodzaj dokumentu, certyfikatu pot...
paragraph    Compostela - rodzaj dokumentu, certyfikatu pot...
Name: 250000, dtype: object

In [4]:
cds_bi_encoder = SentenceTransformer('/content/drive/MyDrive/models/bi-encoder/cds-bi-encoder/cds-bi-encoder-paraphrase-xlm-r-multilingual-v1-2021-01-24_20-05-32')
top_k = 100  

In [5]:
corpus_embeddings_for_cds_bi_encoder = pickle.load( open( '/content/drive/MyDrive/data/corpus_embeddings_for_cds_bi_encoder.pickle', 'rb' ))

In [11]:
corpus_embeddings_for_cds_bi_encoder


array([[-0.09596444,  0.23972692, -0.25753108, ...,  0.03833347,
        -0.13628985, -0.00753768],
       [ 0.14050509,  0.16773991, -0.0584374 , ...,  0.21291272,
        -0.12144812,  0.25173768],
       [ 0.0343454 , -0.1522677 , -0.2871588 , ..., -0.28610498,
        -0.07509118,  0.14915158],
       ...,
       [-0.01577807,  0.06590981,  0.11652289, ...,  0.28648606,
         0.30851927, -0.13399428],
       [ 0.20249091,  0.57107574, -0.07535492, ..., -0.14656283,
         0.21945716,  0.38234153],
       [ 0.17009698,  0.5206695 , -0.06404659, ...,  0.31073496,
         0.03429293, -0.10756476]], dtype=float32)

In [5]:
from sentence_transformers import util
import time
def search(bi_encoder, corpus, corpus_embd,question):
    query = question

    #Encode the query using the bi-encoder and find potentially relevant passages
    start_time = time.time()
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embd, top_k=10)
    hits = hits[0]  # Get the hits for the first query

    #Now, score all retrieved passages with the cross_encoder
    #cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
    #cross_scores = cross_encoder.predict(cross_inp)
    ##Sort results by the cross-encoder scores
    #for idx in range(len(cross_scores)):
    #    hits[idx]['cross-score'] = cross_scores[idx]

   # hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    end_time = time.time()

    #Output of top-5 hits
    print("Input question:", query)
    print("Results (after {:.3f} seconds):".format(end_time - start_time))
    for hit in hits[0:10]:
        index = hit['corpus_id']
        article = corpus.iloc[index]
        print('\nTitle:', article['title'])
        print('Paragraph:', article['paragraph'])
        print('Url:', article['url'])
        print('Index:', index)
    print("\n\n========\n")

# **Badanie działania modelów na zbiorze 250k paragrafów**

ocena algorytmów odbywa się heurystycznie, bazuje na wskazaniu pozycji na których znajduje się odpowiedź


In [7]:
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"darmowy serwer http")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"środki zwalczające infekcje bakteryjne")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"pierwotniak posiadający nibynóżki")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"substancje pozwalające zabarwić ciasto")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 11.601 seconds):

Title: Cache server
Paragraph: Cache server – sposób organizacji pamięci w systemach klient-serwer, w którym dane pobierane z serwera mogą być przechowywane na serwerze pośredniczącym. Przy kolejnym zapytaniu o te same dane pobierane są one z pamięci podręcznej, co eliminuje nadmierną częstotliwość przesyłań w sieci.
Url: http://pl.wikipedia.org/wiki/?curid=478187
Index: 204260

Title: Bitstream Cyberbit
Paragraph: Bitstream nie oferuje już fontów Bitstream Cyberbit ani jako produkt komercyjny, ani jako darmowe do pobrania do zastosowań niekomercyjnych. Dostępne są one do pobrania z serwera Netscape.
Url: http://pl.wikipedia.org/wiki/?curid=1425913
Index: 159202

Title: Apache HTTP Server
Paragraph: Apache – otwarty serwer HTTP dostępny dla wielu systemów operacyjnych (m.in. UNIX, GNU/Linux, BSD, OS X, Microsoft Windows).
Url: http://pl.wikipedia.org/wiki/?curid=8995
Index: 98930

Title: Anelectron
Paragraph: Advanced

### **Efekty działania dla zapytań (model : cds_bi_encoder):**
* "darmowy serwer http" - **3,8**
* "środki zwalczające infekcje bakteryjne" - **1,2,3,4,6,8,10**
* "mikroorganizm posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **1**
* "substancje pozwalające zabarwić ciasto" - **1,3**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **1,3**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **1**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1**
* "drwniane, okrągłe, naczynie do prania" - **7**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**


In [5]:
cds_czywiesz_bi_encoder = SentenceTransformer('/content/drive/MyDrive/models/bi-encoder/cds-czywiesz-bi-encoder/cds-czywiesz-bi-encoder--content-drive-MyDrive-models-cds-bi-encoder-cds-bi-encoder-paraphrase-xlm-r-multilingual-v1-2021-01-24_20-05-32-2021-01-24_22-00-53')
corpus_embeddings_for_cds_czywiesz_bi_encoder = pickle.load( open( '/content/drive/MyDrive/data/corpus_embeddings_for_cds_czywiesz_bi_encoder.pickle', 'rb' ))

In [9]:
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"darmowy serwer http")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"środki zwalczające infekcje bakteryjne")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"pierwotniak posiadający nibynóżki")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"substancje pozwalające zabarwić ciasto")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 0.633 seconds):

Title: ALT Linux
Paragraph: Od wersji 4.0, dostępne jest specjalizowane serwerowe wydanie pod nazwą "Server".System przeznaczony jest zarówno dla architektury x86, jak i dla x86-64.
Url: http://pl.wikipedia.org/wiki/?curid=1557571
Index: 35236

Title: Cache server
Paragraph: Cache server – sposób organizacji pamięci w systemach klient-serwer, w którym dane pobierane z serwera mogą być przechowywane na serwerze pośredniczącym. Przy kolejnym zapytaniu o te same dane pobierane są one z pamięci podręcznej, co eliminuje nadmierną częstotliwość przesyłań w sieci.
Url: http://pl.wikipedia.org/wiki/?curid=478187
Index: 204260

Title: AJAX
Paragraph: Teoretycznie są to wszystkie wymagane elementy, jednak w praktyce używane są jeszcze odpowiednie skrypty funkcjonujące po stronie serwera i współpracujące z bazą danych. Można sobie jednak bez nich poradzić, jeśli wszystkie potrzebne dane zostały już wcześniej wygenerowane (np. zaw

### **Efekty działania dla zapytań (model : cds_czywiesz_bi_encoder):**
* "darmowy serwer http" - **5,7**
* "środki zwalczające infekcje bakteryjne" - **1,2,3,5,8,9,10**
* "pierwotniak posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **2,3**
* "substancje pozwalające zabarwić ciasto" - **1,5,10**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **1,6,7**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **5**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1,6**
* "drwniane, okrągłe, naczynie do prania" - **BRAK**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**

In [6]:
cds_czywiesz_label_bi_encoder = SentenceTransformer('/content/drive/MyDrive/models/bi-encoder/cds-czywiesz-labels-bi-encoder/cds_czywiesz_labels-bi-encoder-bert-base-multilingual-cased-2021-01-24_22-52-29')
corpus_embeddings_for_cds_czywiesz_label_bi_encoder = pickle.load( open( '/content/drive/MyDrive/data/corpus_embeddings_for_cds_czywiesz_labels_bi_encoder.pickle', 'rb' ))

In [7]:
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"darmowy serwer http")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"środki zwalczające infekcje bakteryjne")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"pierwotniak posiadający nibynóżki")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"substancje pozwalające zabarwić ciasto")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 11.457 seconds):

Title: Anelectron
Paragraph: Advanced Electronic Forums (w skrócie AEF) – skrypt forum internetowego. Skrypt jest napisany w języku PHP i wykorzystuje bazę danych MySQL. Udostępniany jest za darmo.
Url: http://pl.wikipedia.org/wiki/?curid=1657123
Index: 87502

Title: Cache server
Paragraph: Cache server – sposób organizacji pamięci w systemach klient-serwer, w którym dane pobierane z serwera mogą być przechowywane na serwerze pośredniczącym. Przy kolejnym zapytaniu o te same dane pobierane są one z pamięci podręcznej, co eliminuje nadmierną częstotliwość przesyłań w sieci.
Url: http://pl.wikipedia.org/wiki/?curid=478187
Index: 204260

Title: Bitstream Cyberbit
Paragraph: Bitstream nie oferuje już fontów Bitstream Cyberbit ani jako produkt komercyjny, ani jako darmowe do pobrania do zastosowań niekomercyjnych. Dostępne są one do pobrania z serwera Netscape.
Url: http://pl.wikipedia.org/wiki/?curid=1425913
Index: 159202

### **Efekty działania dla zapytań (model : cds_czywiesz_label_bi_encoder):**
* "darmowy serwer http" - **7**
* "środki zwalczające infekcje bakteryjne" - **1,2,4,5,9**
* "pierwotniak posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **5**
* "substancje pozwalające zabarwić ciasto" - **5**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **1**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **2**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1**
* "drwniane, okrągłe, naczynie do prania" - **BRAK**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**